In [ ]:
!pip install --quiet --upgrade torch torchvision torchaudio

In [ ]:
!pip install --quiet --upgrade transformers

In [ ]:
!pip install --quiet --upgrade bitsandbytes accelerate

In [ ]:
!pip install --quiet --upgrade sentencepiece

In [ ]:
!pip install --quiet --upgrade protobuf

In [1]:
import torch
from transformers import BitsAndBytesConfig

# model_name = "GritLM/GritLM-7B"
# model_id = "mistralai/Mistral-7B-Instruct-v0.3"
# model_name = "google/gemma-1.1-7b-it"

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "meta-llama/Meta-Llama-3-70B-Instruct"
# model_name = "nvidia/Llama3-ChatQA-1.5-8B"

## 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

## 8-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=False
# )

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    # "torch_dtype": "auto",
    # "quantization_config": bnb_config,
}

device = "cuda"

In [2]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_id)
streamer = TextStreamer(tokenizer)

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs=model_kwargs,
    device=device,
    streamer=streamer
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import utils
utils.print_model_info(pipe.model)

Model size : 15316 MB
Model is quantized : False
Model on GPU : True


In [4]:
conversation = [
    {"role": "assistant", "content": "what is your name?"},
    {"role": "user", "content": "my name is Jack"},
    {"role": "assistant", "content": "how are you today, Jack?"},
    {"role": "user", "content": "I'm great, thanks" },
    {"role": "assistant", "content": "are you a student or employed?"},
    {"role": "user", "content": "I am a farmer"},
    {"role": "assistant", "content": "are you seeing someone or are you married?"},
    {"role": "user", "content": "yes, my wife's name is Jill"},
    {"role": "assistant", "content": "how long have you been married?"},
    {"role": "user", "content": "15 years"},
    {"role": "assistant", "content": "do you have any children?"},
    {"role": "user", "content": "yes, two"},
    {"role": "assistant", "content": "what are their names?"},
    {"role": "user", "content": "Jack and Jill"},
    {"role": "assistant", "content": "how old are they?"},
    {"role": "user", "content": "10 and 12, respectively"},
]

In [5]:
prompt = """
Summarize information about the user from the following coversation:
"""

for message in conversation:
    prompt += f"{message['role']}: {message['content']}\n"

messages = [
    {"role": "user", "content": prompt}
]

sequences = pipe(
    messages,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.1,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=2048
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Summarize information about the user from the following coversation:
assistant: what is your name?
user: my name is Jack
assistant: how are you today, Jack?
user: I'm great, thanks
assistant: are you a student or employed?
user: I am a farmer
assistant: are you seeing someone or are you married?
user: yes, my wife's name is Jill
assistant: how long have you been married?
user: 15 years
assistant: do you have any children?
user: yes, two
assistant: what are their names?
user: Jack and Jill
assistant: how old are they?
user: 10 and 12, respectively<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here is a summary of the information about the user, Jack:

* Name: Jack
* Occupation: Farmer
* Marital status: Married to Jill
* Length of marriage: 15 years
* Children: Two, named Jack and Jill
* Ages of children: 10 and 12, respectively<|eot_id|>


In [8]:
prompt = """
You are an AI assistant that identifies facts about the user from chat conversations.
You list facts in a JSON array, with each fact in a dictionary with keys fact and confidence_score.
The confidence score must be accurate.
List all discrete facts about the user from the following chat conversation:
"""

for message in conversation:
    prompt += f"{message['role']}: {message['content']}\n"

messages = [
    {"role": "user", "content": prompt}
]

sequences = pipe(
    messages,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are an AI assistant that identifies facts about the user from chat conversations.
You list facts in a JSON array, with each fact in a dictionary with keys fact and confidence_score.
The confidence score must be accurate.
List all discrete facts about the user from the following chat conversation:
assistant: what is your name?
user: my name is Jack
assistant: how are you today, Jack?
user: I'm great, thanks
assistant: are you a student or employed?
user: I am a farmer
assistant: are you seeing someone or are you married?
user: yes, my wife's name is Jill
assistant: how long have you been married?
user: 15 years
assistant: do you have any children?
user: yes, two
assistant: what are their names?
user: Jack and Jill
assistant: how old are they?
user: 10 and 12, respectively<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here are the discrete facts about the user from the chat conversation:

```
[
  {
    "fact": "The 

In [9]:
prompt = """
You are an AI assistant that identifies facts about the user from chat conversations.
You list facts in a JSON array, with each fact in a dictionary with keys subject, predicate, object, and confidence_score.
The confidence score must be accurate.
List all discrete facts about the user from the following chat conversation:
"""

for message in conversation:
    prompt += f"{message['role']}: {message['content']}\n"

messages = [
    {"role": "user", "content": prompt}
]

sequences = pipe(
    messages,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are an AI assistant that identifies facts about the user from chat conversations.
You list facts in a JSON array, with each fact in a dictionary with keys subject, predicate, object, and confidence_score.
The confidence score must be accurate.
List all discrete facts about the user from the following chat conversation:
assistant: what is your name?
user: my name is Jack
assistant: how are you today, Jack?
user: I'm great, thanks
assistant: are you a student or employed?
user: I am a farmer
assistant: are you seeing someone or are you married?
user: yes, my wife's name is Jill
assistant: how long have you been married?
user: 15 years
assistant: do you have any children?
user: yes, two
assistant: what are their names?
user: Jack and Jill
assistant: how old are they?
user: 10 and 12, respectively<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here are the discrete facts about the user from the chat conversation:

```
